# Pyrfume data processing pipeline for Bushdid et al, 2014

In [1]:
import pandas as pd
from pyrfume.odorants import get_cids, from_cids

### Supplementary Table S1 containing information about the molecules

In [2]:
# Load and trim white space
s1 = pd.read_csv('tableS1.csv', encoding='latin1').iloc[:, :4].dropna()
s1['Odorant name'] = s1['Odorant name'].apply(str.strip)
s1.head()

,Odorant name,C.A.S.,% odorant,Solvent
0,"(3S)-3,7-dimethyloct-6-en-1-ol",7540-51-4,1.000,"1,2-propanediol"
1,(methyldisulfanyl) methane,624-92-0,0.025,"1,2-propanediol"
2,1-hexanol,111-27-3,1.000,mineral oil
3,1-Isopropyl-4-methylbenzene,99-87-6,0.500,"1,2-propanediol"
4,1-octen-3-ol,3391-86-4,0.040,mineral oil


In [3]:
# Get PubChem IDs from CAS registry numbers
cids = get_cids(s1['C.A.S.'])

  0%|          | 0/128 [00:00<?, ?it/s]

Multiple CIDs for 87-44-5: [5281515, 26318, 5281522, 6887, 5322111, 5354499, 6429274]
Multiple CIDs for 17369-59-4: [5373603, 28500, 6259976]
Multiple CIDs for 5655-61-8: [93009, 6448, 44630108, 442460, 12097317, 6950274]
Multiple CIDs for 18172-67-3: [440967, 14896, 6992019, 10290825, 12212059, 12212060, 24848167]
Could not find 110-01-1


In [4]:
# Which molecules could not be mapped to uniqye PubChem IDs?
s1[s1['C.A.S.'].isin(['87-44-5', '17369-59-4', '5655-61-8', '18172-67-3', '110-01-1'])]

,Odorant name,C.A.S.,% odorant,Solvent
35,caryophyllene,87-44-5,15.0000,"1,2-propanediol"
36,celeriax,17369-59-4,0.2000,"1,2-propanediol"
75,iso-bornyl acetate,5655-61-8,5.0000,mineral oil
80,laevo-beta-pinene,18172-67-3,10.0000,"1,2-propanediol"
119,thiolane,110-01-1,0.0005,"1,2-propanediol"


In [5]:
# Add those CIDs manually
cids.update({
    '17369-49-4': 5373603, # celeriax is 3-propylidene phthalide
    '87-44-5': 5281515, # caryophyllene is beta-caryophyllene
    '5655-61-8': 6448, # Assumed racemic mixture for iso-bornyl acetate
    '18172-67-3': 440967, # laevo-beta-pinene is (-)-beta-pinene
    '110-01-1': 1127, # C.A.S. for thiolane was wrong in source file (Excel error)
})
cids = pd.Series(cids, name='CID').astype(int)

In [6]:
# Add the CIDs into the Table S1 dataframe
s1 = s1.join(cids, on='C.A.S.').set_index('CID')
s1 = s1.replace('110-01-1', '110-01-0') # Fix thiolane CID
s1.head()

,Odorant name,C.A.S.,% odorant,Solvent
CID,,,,
7793,"(3S)-3,7-dimethyloct-6-en-1-ol",7540-51-4,1.000,"1,2-propanediol"
12232,(methyldisulfanyl) methane,624-92-0,0.025,"1,2-propanediol"
8103,1-hexanol,111-27-3,1.000,mineral oil
7463,1-Isopropyl-4-methylbenzene,99-87-6,0.500,"1,2-propanediol"
18827,1-octen-3-ol,3391-86-4,0.040,mineral oil


In [7]:
# Get standardized information for these molecules
info = from_cids(cids.values)

  0%|          | 0/2 [00:00<?, ?it/s]

Retrieving 0 through 99
Retrieving 100 through 128


In [8]:
# Join data and save.
molecules = pd.DataFrame(info).set_index('CID').join(s1)
molecules.to_csv('molecules.csv')

### Supplementary Table S2 containing information about the mixtures and the triangle tests that used them

In [9]:
s2 = pd.read_csv('tableS2.csv', encoding='latin1')
s2 = s2[s2['Test UID'].str.isnumeric() > 0]  # Keep only numeric Test UIDs
s2['Test UID'] = s2['Test UID'].astype(int)

In [10]:
# Extract only the trial results for each Test UID and subject
behavior = pd.melt(s2, id_vars=['Test UID'], value_vars=['subject %d' % i for i in range(1, 27)], var_name='Subject', value_name='Correct')
behavior = behavior.replace({'subject %d' % i: i for i in range(1, 27)})
behavior = behavior.dropna().set_index(['Test UID', 'Subject']).sort_index()
behavior = behavior.replace({'wrong': False, 'right': True}).astype(bool)

# Save behavioral data
behavior.to_csv('behavior.csv')
behavior.head()

Correct
Test UID Subject         
1        1          False
         2           True
         3          False
         4           True
         5           True

In [12]:
# Get all the trial metadata (including mixture composition)
mixtures = s2.loc[:, :'subject 1'].iloc[:, :-1]
mixtures = mixtures.rename(columns={'Unnamed: %d' % i: 'Molecule %d' % (i-5) for i in range(6, 36)})

# Replace raw odorant names with CIDs
cid_map = {v.lower():k for k,v in molecules['Odorant name'].items()}
mol_columns = ["Molecule %d" % i for i in range(1, 31)]
mixtures[mol_columns] = mixtures[mol_columns].apply(lambda x: x.str.lower()).fillna(0)
# Manual fixes
mixtures = mixtures.replace(
    {'\xa01-isopropyl-4-methylbenzene': '1-isopropyl-4-methylbenzene',
     '4-methyl-3-penten-2-one': '4-methylpent-3-en-2-one'})
mixtures = mixtures.replace(cid_map)
mixtures[mol_columns] = mixtures[mol_columns].astype(int)

# Cleanup other names
mixtures['Stimulus dilution'] = mixtures['Stimulus dilution'].replace({'1/4': 0.25, '1/2': 0.5, 'not diluted': 1})
mixtures[['Components in mixtures', 'Components that differ']] = mixtures[['Components in mixtures', 'Components that differ']].astype(int)
mixtures = mixtures.set_index(['Test UID', 'Answer']).sort_index()

# Sanity check
for i, answer in enumerate(mixtures.index.get_level_values('Answer')):
    if i % 3 == 0:
        assert answer=='right'
    if i % 3 != 0:
        assert answer=='wrong'
mixtures.head(9)

Components in mixtures  Components that differ  \
Test UID Answer                                                   
1        right                       30                      30   
         wrong                       30                      30   
         wrong                       30                      30   
2        right                       10                       4   
         wrong                       10                       4   
         wrong                       10                       4   
3        right                       10                       1   
         wrong                       10                       1   
         wrong                       10                       1   

                 % mixture overlap  Stimulus dilution  Molecule 1  Molecule 2  \
Test UID Answer                                                                 
1        right                 0.0               1.00       12232        7731   
         wrong                 0.0               0.25      440917     5281515   
         wrong                 0.0               0.50      440917     5281515   
2        right                60.0               0.50        3314       62433   
         wrong                60.0               0.25        3314       62433   
         wrong                60.0               1.00        3314       62433   
3        right                90.0               0.25        8797        2879   
         wrong                90.0               0.50        8797        2879   
         wrong                90.0               1.00        8797        2879   

                 Molecule 3  Molecule 4  Molecule 5  Molecule 6  ...  \
Test UID Answer                                                  ...   
1        right         7888        7966        7848       18827  ...   
         wrong         8030        3314       31272         798  ...   
         wrong         8030        3314       31272         798  ...   
2        right      5281515        7749        6561       12178  ...   
         wrong      5281515        7749        6561       12178  ...   
         wrong      5281515        7749        6561       12178  ...   
3        right         7888        6501      443158       22873  ...   
         wrong         7888        6501      443158       22873  ...   
         wrong         7888        6501      443158       22873  ...   

                 Molecule 21  Molecule 22  Molecule 23  Molecule 24  \
Test UID Answer                                                       
1        right          8103         7344         8051         7921   
         wrong          6561        22386        11509       443162   
         wrong          6561        22386        11509       443162   
2        right             0            0            0            0   
         wrong             0            0            0            0   
         wrong             0            0            0            0   
3        right             0            0            0            0   
         wrong             0            0            0            0   
         wrong             0            0            0            0   

                 Molecule 25  Molecule 26  Molecule 27  Molecule 28  \
Test UID Answer                                                       
1        right           460         7583        11002        12367   
         wrong          8797        16666         7749         7793   
         wrong          8797        16666         7749         7793   
2        right             0            0            0            0   
         wrong             0            0            0            0   
         wrong             0            0            0            0   
3        right             0            0            0            0   
         wrong             0            0            0            0   
         wrong             0            0            0            0   

                 Molecule 29  

In [13]:
# Save mixtures data
mixtures.to_csv('mixtures.csv')